<a href="https://colab.research.google.com/github/hardik0/Deep-Learning-with-GoogleColab/blob/master/VideoColorizerColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#◢ DeOldify - Colorize your own Videos/Images!


_FYI: This notebook is intended as a tool to colorize GIFs and short Videos and Images, if you are trying to convert longer video you may hit the limit on processing space. Running the Jupyter notebook on your own machine is recommended (and faster) for larger video sizes._

####**Credits:**

Big special thanks to:

**Robert Bell** for all his work on the video Colab notebook, and paving the way to video in DeOldify!

**Matt Robinson** and **María Benavente** for pioneering the DeOldify image colab notebook.

**Dana Kelley** for doing things, breaking stuff & having an opinion on everything.

In [0]:
from google.colab import drive
drive.mount('/content/drive')



---


#◢ Verify Correct Runtime Settings

**<font color='#FF000'> IMPORTANT </font>**

In the "Runtime" menu for the notebook window, select "Change runtime type." Ensure that the following are selected:
* Runtime Type = Python 3
* Hardware Accelerator = GPU 

Your instance must have following gpu to process and render the video
* Tesla T4 
* Tesla P100


In [0]:
!nvidia-smi

Tue Nov 19 03:15:03 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.50       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

**Check GPU Type**

In [0]:
import pynvml


pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)
device_name = pynvml.nvmlDeviceGetName(handle)
print(device_name)

if device_name != b'Tesla T4' and device_name != b'Tesla P100-PCIE-16GB':
  raise Exception("""
    Unfortunately this instance does not have a Tesla T4 /Tesla P100 GPU.
    
    Please make sure you've configured Colab to request a GPU instance type.
    
    Sometimes Colab allocates a Tesla K80 instead of a T4. Resetting the instance.

    If you get a K80 GPU, try Runtime -> Reset all runtimes... 
    """)
else:
  print('Woo! You got the right kind of GPU!')

b'Tesla P100-PCIE-16GB'
Woo! You got the right kind of GPU!


In [0]:
from os import path
import torch

#◢ Git clone and install DeOldify

In [0]:
!git clone https://github.com/jantic/DeOldify.git DeOldify

In [0]:
cd DeOldify

/content/DeOldify


#◢ Setup

In [0]:
!pip install -r requirements.txt

In [0]:
import fastai
from deoldify.visualize import *
from pathlib import Path
torch.backends.cudnn.benchmark=True

In [0]:
!mkdir 'models'
#Download Pretrained Weights for video
!wget https://www.dropbox.com/s/336vn9y4qwyg9yz/ColorizeVideo_gen.pth?dl=0 -O ./models/ColorizeVideo_gen.pth
#Download Pretrained Weights for image
!wget https://www.dropbox.com/s/zkehq1uwahhbc2o/ColorizeArtistic_gen.pth?dl=0 -O ./models/ColorizeArtistic_gen.pth

In [0]:
colorizer = get_video_colorizer()
colorizer_img = get_image_colorizer(artistic=True)

#◢ Instructions

### source_url (video)
Type in a url hosting a video from YouTube, Imgur, Twitter, Reddit, Vimeo, etc.  Many sources work!  GIFs also work.  Full list here: https://ytdl-org.github.io/youtube-dl/supportedsites.html NOTE: If you want to use your own video, upload it first to a site like YouTube. 

### source_url (image)
Type in a url to a direct link of an image.  Usually that means they'll end in .png, .jpg, etc. NOTE: If you want to use your own image, upload it first to a site like Imgur. 

### render_factor
The default value of 21(for Vidoe) and 35(for image) has been carefully chosen and should work -ok- for most scenarios (but probably won't be the -best-). This determines resolution at which the color portion of the video is rendered. Lower resolution will render faster, and colors also tend to look more vibrant. Older and lower quality film in particular will generally benefit by lowering the render factor. Higher render factors are often better for higher quality videos and inconsistencies (flashy render) will generally be reduced, but the colors may get slightly washed out. 

### How to Download a Copy
Simply right click on the displayed video/image and click "Save video as..."!

## Pro Tips
1. If a video takes a long time to render and you're wondering how well the frames will actually be colorized, you can preview how well the frames will be rendered at each render_factor by using the code at the bottom. Just stop the video rendering by hitting the stop button on the cell, then run that bottom cell under "See how well render_factor values perform on a frame here". It's not perfect and you may still need to experiment a bit especially when it comes to figuring out how to reduce frame inconsistency.  But it'll go a long way in narrowing down what actually works.
2. If videos are taking way too much time for your liking, running the Jupyter notebook VideoColorizer.ipynb on your own machine (with DeOldify installed) will generally be much faster (as long as you have the hardware for it). 
3.You can evaluate how well the image is rendered at each render_factor by using the code at the bottom (that cell under "See how well render_factor values perform on a frame here").

## Troubleshooting
The video player may wind up not showing up, in which case- make sure to wait for the Jupyter cell to complete processing first (the play button will stop spinning).  Then follow these alternative download instructions

1. In the menu to the left, click Files
2. If you don't see the 'DeOldify' folder, click "Refresh"
3. By default, rendered video will be in /DeOldify/video/result/

If a video you downloaded doesn't play, it's probably because the cell didn't complete processing and the video is in a half-finished state.

If you get a 'CUDA out of memory' error, you probably have the render_factor too high. The max is 44 on 11GB video cards.

#◢ Download Youtube Videos
make directory and cd into it

In [0]:
%cd ..
%mkdir youtube_videos
%cd youtube_videos

/content
/content/youtube_videos


Download all playlists of YouTube channel/user keeping each playlist in separate directory & best mp4 format available or any other best if no mp4 available

In [0]:
!youtube-dl -f 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best' -o '%(uploader)s/%(playlist)s/%(playlist_index)s - %(title)s.%(ext)s' --restrict-filenames -v https://www.youtube.com/user/Tommydan333

 Download  only video description to a .description file (optional)

In [0]:
!youtube-dl --skip-download --youtube-skip-dash-manifest --write-description -o '%(uploader)s/%(playlist)s/%(playlist_index)s - %(title)s.%(ext)s' --restrict-filenames -v https://www.youtube.com/user/Tommydan333

In [0]:
!ls Tommydan333/Uploads_from_Tommydan333/

Zip downloaded .discription file

In [0]:
!zip -r all_video_desc.zip Tommydan333/Uploads_from_Tommydan333/

In [0]:
%ls '../youtube_videos/Tommydan333/'

Backup Videos to Google Drive (optional)

In [0]:
%cp -r '/Tommydan333/.' '../drive/My Drive/Colab Notebooks/Old_BackupVideos/' 

'/content/DeOldify'

Copy Videos from Google Drive - You can use the drag and drop method to copy data using files manager. (to enable **View > Table of contents**)

In [0]:
%mkdir video
%mkdir video/source
%cp -r '../drive/My Drive/Colab Notebooks/Old_BackupVideos/.' video/source/
#%ls 'video/source/'

#◢ Colorize!!! - Image/Photo

### Colorize from URL

In [0]:
source_url = '' #@param {type:"string"}
render_factor = 34  #@param {type: "slider", min: 7, max: 45}

if source_url is not None and source_url !='':
    image_path = colorizer_img.plot_transformed_image_from_url(url=source_url, render_factor=render_factor, compare=True)
    show_image_in_notebook(image_path)
else:
    print('Provide an image url and try again.')

## See how well render_factor values perform on the image here

In [0]:
for i in range(10,46,2):
    colorizer_img.plot_transformed_image('test_images/image.png', render_factor=i, display_render_factor=True, figsize=(7,7))

#◢ Colorize!!! - Video 

### Colorize from URL

In [0]:
source_url = '' #@param {type:"string"}
render_factor = 21  #@param {type: "slider", min: 5, max: 44}

if source_url is not None and source_url !='':
    video_path = colorizer.colorize_from_url(source_url, 'video.mp4', render_factor)
    #show_video_in_notebook(video_path)
else:
    print('Provide a video url and try again.')

### Colorize from File

In [0]:
class color:
  BLUE = '\033[94m'
  GREEN = '\033[92m'
  RED = '\033[91m'
  BOLD = '\033[1m'   
  UNDERLINE = '\033[4m'
  END = '\033[0m' 
  #The above code is just for fun only!

import os
import shutil
from os import listdir
from os.path import isfile, join

render_factor = 21

#This is the default directory, first you have to copy the video here
old_video_source = 'video/source/'

 
fileName = []
i = []  
fileNames = [f for f in listdir(old_video_source) if isfile(join(old_video_source, f))]

for fileName in fileNames:
  
  try:
    
    #Video proccessing & rendering
    print(color.BOLD + str(fileName) + color.END + color.BLUE + ' ready for proccessig.' + color.END)
    video_path = colorizer.colorize_from_file_name(str(fileName), render_factor)  
    print(color.GREEN + 'Video rendering done, Now ' + color.END + color.BOLD + fileName + color.END + ' file ready for copy.')
    
    #Copying file  
    build_video_dir = 'video/result/'
    new_build_video_path = build_video_dir + str(fileName)
    target_dir = '../drive/My Drive/Colab Notebooks/Old_ColorizeVideos'

    assert not os.path.isabs(new_build_video_path)
    target = os.path.join(target_dir, os.path.dirname(new_build_video_path))

    # Create the folders if not already exists
    #os.makedirs(target_dir)

    # adding exception handling
    try:
      shutil.copy(new_build_video_path, target_dir)
      print(color.BOLD + fileName + color.RED + " Successfully Copied to "  + color.END + target_dir + "\n")
          
    except IOError as e:      
      print("Unable to copy file. %s" % e)
   
  except:
    print("Unexpected error:", sys.exc_info())
    

### See how well render_factor values perform on a frame here

In [0]:
for i in range(10,45,2):
    colorizer.vis.plot_transformed_image('video/bwframes/video/00001.jpg', render_factor=i, display_render_factor=True, figsize=(5,5))

---
#⚙ Recommended video and gif sources 
* [/r/Nickelodeons/](https://www.reddit.com/r/Nickelodeons/)
* [r/silentmoviegifs](https://www.reddit.com/r/silentmoviegifs/)
* https://twitter.com/silentmoviegifs 

---
#⚙ Recommended image sources 
* [/r/TheWayWeWere](https://www.reddit.com/r/TheWayWeWere/)